In [49]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from sklearn.impute import SimpleImputer

import warnings; warnings.simplefilter('ignore')

In [50]:
df = pd. read_csv('/kaggle/input/top-5000-albums-of-all-time-rateyourmusiccom/rym_top_5000_all_time.csv')
df.head(10)

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1.0,OK Computer,Radiohead,16 June 1997,"Alternative Rock, Art Rock","melancholic, anxious, futuristic, alienation, ...",4.23,"70,382",1531
1,2.0,Wish You Were Here,Pink Floyd,12 September 1975,"Progressive Rock, Art Rock","melancholic, atmospheric, progressive, male vo...",4.29,"48,662",983
2,3.0,In the Court of the Crimson King,King Crimson,10 October 1969,"Progressive Rock, Art Rock","fantasy, epic, progressive, philosophical, com...",4.30,"44,943",870
3,4.0,Kid A,Radiohead,3 October 2000,"Art Rock, Experimental Rock, Electronic","cold, melancholic, futuristic, atmospheric, an...",4.21,"58,590",734
4,5.0,To Pimp a Butterfly,Kendrick Lamar,15 March 2015,"Conscious Hip Hop, West Coast Hip Hop, Jazz Rap","political, conscious, poetic, protest, concept...",4.27,"44,206",379
5,6.0,Loveless,My Bloody Valentine,4 November 1991,"Shoegaze, Noise Pop","noisy, ethereal, atmospheric, romantic, dense,...",4.24,"49,887",1223
6,7.0,The Dark Side of the Moon,Pink Floyd,23 March 1973,"Art Rock, Progressive Rock","philosophical, atmospheric, introspective, exi...",4.20,"57,622",1549
7,8.0,Abbey Road,The Beatles,26 September 1969,Pop Rock,"melodic, warm, male vocals, bittersweet, summe...",4.25,"44,544",961
8,9.0,The Velvet Underground & Nico,The Velvet Underground & Nico,12 March 1967,"Art Rock, Experimental Rock","drugs, sexual, raw, urban, noisy, nihilistic, ...",4.23,"45,570",929
9,10.0,The Rise and Fall of Ziggy Stardust and the Sp...,David Bowie,16 June 1972,"Glam Rock, Pop Rock","science fiction, melodic, anthemic, concept al...",4.26,"39,501",721


In [51]:
row_index = 7
df['Number of Ratings'] = df['Number of Ratings'].str.replace(',', '')

In [52]:
vote_counts = df[df['Number of Ratings'].notnull()]['Number of Ratings'].astype('int')
vote_averages = df[df['Average Rating'].notnull()]['Average Rating'].astype('int')

C = vote_averages.mean()

C

3.037

In [53]:
m = vote_counts.quantile(0.95)
m

15969.50000000001

In [54]:
df['Year'] = df['Release Date'].str[-4:]

df

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews,Year
0,1.0,OK Computer,Radiohead,16 June 1997,"Alternative Rock, Art Rock","melancholic, anxious, futuristic, alienation, ...",4.23,70382,1531,1997
1,2.0,Wish You Were Here,Pink Floyd,12 September 1975,"Progressive Rock, Art Rock","melancholic, atmospheric, progressive, male vo...",4.29,48662,983,1975
2,3.0,In the Court of the Crimson King,King Crimson,10 October 1969,"Progressive Rock, Art Rock","fantasy, epic, progressive, philosophical, com...",4.30,44943,870,1969
3,4.0,Kid A,Radiohead,3 October 2000,"Art Rock, Experimental Rock, Electronic","cold, melancholic, futuristic, atmospheric, an...",4.21,58590,734,2000
4,5.0,To Pimp a Butterfly,Kendrick Lamar,15 March 2015,"Conscious Hip Hop, West Coast Hip Hop, Jazz Rap","political, conscious, poetic, protest, concept...",4.27,44206,379,2015
...,...,...,...,...,...,...,...,...,...,...
4995,4996.0,Gracious!,Gracious,17 July 1970,"Progressive Rock, Symphonic Prog",NaN,3.69,565,32,1970
4996,4997.0,Knights of the Cross,Grave Digger,18 May 1998,"Power Metal, Heavy Metal","history, male vocals, heavy, concept album, an...",3.67,562,11,1998
4997,4998.0,(Listen For) The Rag and Bone Man,And Also the Trees,12 November 2007,Art Rock,"atmospheric, poetic, dark, male vocals, ominou...",3.68,524,15,2007
4998,4999.0,Hymnen für elektronische und konkrete Klänge,Studio für elektronische Musik des Westdeutsch...,1969,"Musique concrète, Modern Classical, Electronic","avant-garde, sampling, dense, complex, scary, ...",3.65,536,11,1969


In [55]:
qualified = df[(df['Number of Ratings'].astype('int') >= m) & (df['Number of Ratings'].notnull()) & (df['Average Rating'].notnull())][['Album', 'Artist Name', 'Year', 'Number of Ratings', 'Average Rating', 'Descriptors', 'Genres']]
qualified['Number of Ratings'] = qualified['Number of Ratings'].astype('int')
qualified['Average Rating'] = qualified['Average Rating'].astype('float')
qualified.shape

qualified.head(10)

,Album,Artist Name,Year,Number of Ratings,Average Rating,Descriptors,Genres
0,OK Computer,Radiohead,1997,70382,4.23,"melancholic, anxious, futuristic, alienation, ...","Alternative Rock, Art Rock"
1,Wish You Were Here,Pink Floyd,1975,48662,4.29,"melancholic, atmospheric, progressive, male vo...","Progressive Rock, Art Rock"
2,In the Court of the Crimson King,King Crimson,1969,44943,4.30,"fantasy, epic, progressive, philosophical, com...","Progressive Rock, Art Rock"
3,Kid A,Radiohead,2000,58590,4.21,"cold, melancholic, futuristic, atmospheric, an...","Art Rock, Experimental Rock, Electronic"
4,To Pimp a Butterfly,Kendrick Lamar,2015,44206,4.27,"political, conscious, poetic, protest, concept...","Conscious Hip Hop, West Coast Hip Hop, Jazz Rap"
5,Loveless,My Bloody Valentine,1991,49887,4.24,"noisy, ethereal, atmospheric, romantic, dense,...","Shoegaze, Noise Pop"
6,The Dark Side of the Moon,Pink Floyd,1973,57622,4.20,"philosophical, atmospheric, introspective, exi...","Art Rock, Progressive Rock"
7,Abbey Road,The Beatles,1969,44544,4.25,"melodic, warm, male vocals, bittersweet, summe...",Pop Rock
8,The Velvet Underground & Nico,The Velvet Underground & Nico,1967,45570,4.23,"drugs, sexual, raw, urban, noisy, nihilistic, ...","Art Rock, Experimental Rock"
9,The Rise and Fall of Ziggy Stardust and the Sp...,David Bowie,1972,39501,4.26,"science fiction, melodic, anthemic, concept al...","Glam Rock, Pop Rock"


In [56]:
def weighted_rating(x):
    v = x['Number of Ratings']
    R = x['Average Rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [57]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [58]:
qualified.head(15)

,Album,Artist Name,Year,Number of Ratings,Average Rating,Descriptors,Genres,wr
0,OK Computer,Radiohead,1997,70382,4.23,"melancholic, anxious, futuristic, alienation, ...","Alternative Rock, Art Rock",4.009371
1,Wish You Were Here,Pink Floyd,1975,48662,4.29,"melancholic, atmospheric, progressive, male vo...","Progressive Rock, Art Rock",3.980402
2,In the Court of the Crimson King,King Crimson,1969,44943,4.30,"fantasy, epic, progressive, philosophical, com...","Progressive Rock, Art Rock",3.968878
3,Kid A,Radiohead,2000,58590,4.21,"cold, melancholic, futuristic, atmospheric, an...","Art Rock, Experimental Rock, Electronic",3.958761
5,Loveless,My Bloody Valentine,1991,49887,4.24,"noisy, ethereal, atmospheric, romantic, dense,...","Shoegaze, Noise Pop",3.948285
6,The Dark Side of the Moon,Pink Floyd,1973,57622,4.20,"philosophical, atmospheric, introspective, exi...","Art Rock, Progressive Rock",3.947627
4,To Pimp a Butterfly,Kendrick Lamar,2015,44206,4.27,"political, conscious, poetic, protest, concept...","Conscious Hip Hop, West Coast Hip Hop, Jazz Rap",3.942784
7,Abbey Road,The Beatles,1969,44544,4.25,"melodic, warm, male vocals, bittersweet, summe...",Pop Rock,3.929890
8,The Velvet Underground & Nico,The Velvet Underground & Nico,1967,45570,4.23,"drugs, sexual, raw, urban, noisy, nihilistic, ...","Art Rock, Experimental Rock",3.920417
9,The Rise and Fall of Ziggy Stardust and the Sp...,David Bowie,1972,39501,4.26,"science fiction, melodic, anthemic, concept al...","Glam Rock, Pop Rock",3.907908


In [59]:
s = df.apply(lambda x: pd.Series(x['Genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = df.drop('Genres', axis=1).join(s)

s

0                            Alternative Rock, Art Rock
1                            Progressive Rock, Art Rock
2                            Progressive Rock, Art Rock
3               Art Rock, Experimental Rock, Electronic
4       Conscious Hip Hop, West Coast Hip Hop, Jazz Rap
                             ...                       
4995                   Progressive Rock, Symphonic Prog
4996                           Power Metal, Heavy Metal
4997                                           Art Rock
4998     Musique concrète, Modern Classical, Electronic
4999                              Smooth Soul, Pop Soul
Name: genre, Length: 5000, dtype: object

In [60]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['Number of Ratings'].notnull()]['Number of Ratings'].astype('int')
    vote_averages = df[df['Average Rating'].notnull()]['Average Rating'].astype('float')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['Number of Ratings'].astype('int') >= m) & (df['Number of Ratings'].notnull()) & (df['Average Rating'].notnull())][['Album', 'Year', 'Number of Ratings', 'Average Rating', 'Descriptors']]
    qualified['Number of Ratings'] = qualified['Number of Ratings'].astype('int')
    qualified['Average Rating'] = qualified['Average Rating'].astype('float')
    
    qualified['wr'] = qualified.apply(lambda x: (x['Number of Ratings']/(x['Number of Ratings']+m) * x['Average Rating']) + (m/(m+x['Number of Ratings']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [61]:
build_chart('Heavy Metal').head()

,Album,Year,Number of Ratings,Average Rating,Descriptors,wr
51,Master of Reality,1971,24271,4.12,"heavy, dark, male vocals, rhythmic, sombre, no...",4.031830
167,Powerslave,1984,15660,3.98,"energetic, epic, history, melodic, male vocals...",3.909302
171,Painkiller,1990,10710,4.00,"energetic, male vocals, fantasy, epic, science...",3.902208
227,Seventh Son of a Seventh Son,1988,12006,3.96,"epic, fantasy, melodic, male vocals, progressi...",3.884910
480,Screaming for Vengeance,1982,8460,3.88,"energetic, male vocals, anthemic, rebellious, ...",3.829072


## Content Based Recommender

#### Combining Descriptors, Genres and Artist Name to make a joint metadata

In [62]:
df['metadata'] = df['Descriptors'] + df['Genres'] + df['Artist Name']

#### Filling missing values in the 'metadata' column with an empty string. 

In [63]:
imputer = SimpleImputer(strategy='constant', fill_value='') 
data = imputer.fit_transform(np.array(df['metadata']).reshape(-1, 1)).flatten()

#### Transforming the input text data into a sparse matrix of TF-IDF features 

In [64]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 1), min_df=0, stop_words=None)
tfidf_matrix = tf.fit_transform(data)

#### Using the linear_kernel function to efficiently compute the dot product between each pair of rows, resulting in a symmetric matrix (cosine_sim).

### Cosine Similarity:

cosine(x,y)=x.y⊺||x||.||y||

In [65]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [66]:
cosine_sim[0]

array([1.        , 0.15257732, 0.13852032, ..., 0.03241526, 0.        ,
       0.0063652 ])

In [67]:
df = df.reset_index()
titles = df['Album']
indices = pd.Series(df.index, index=df['Album'])

#### Function for returning the most similar albums based on cosine similarity score. Limit is 30.

In [68]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    
    return titles.iloc[movie_indices]

In [69]:
get_recommendations('In Rainbows').head(10)

186             The Bends
0             OK Computer
343              Amnesiac
1201    Hail to the Thief
1792              Marbles
1156              Takk...
4277         Chelsea Girl
643               Endless
1712        Wrecking Ball
3825      The Ideal Crash
Name: Album, dtype: object

In [70]:
get_recommendations('To Pimp a Butterfly').head(10)

3513                                       Section.80
2615                             Untitled Unmastered.
16                             good kid, m.A.A.d city
3116                          The Devil Made Me Do It
2315                                 Steal This Album
294     It Takes a Nation of Millions to Hold Us Back
4064                          Sleeping With the Enemy
4421                                            Damn.
3000                                 Genocide & Juice
1408                                          Absence
Name: Album, dtype: object